In [3]:
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx

import networkx as nx
import numpy as np
import planarity
from tqdm import tqdm

import networkx as nx
import torch

# load the data


def load_data():
    # 영업일에만 존재하는 데이터
    df_open = pd.read_csv('open.csv',index_col=0, thousands=',')
    df_open.index = pd.to_datetime(df_open.index)
    df_high = pd.read_csv('high.csv',index_col=0, thousands=',')
    df_high.index = pd.to_datetime(df_high.index)
    df_low = pd.read_csv('low.csv',index_col=0, thousands=',')
    df_low.index = pd.to_datetime(df_low.index)
    df_close = pd.read_csv('close.csv',index_col=0, thousands=',')
    df_close.index = pd.to_datetime(df_close.index)
    df_vol = pd.read_csv('volume.csv',index_col=0, thousands=',')
    df_vol.index = pd.to_datetime(df_vol.index)
    # 아래 시가총액 데이터는 매일 존재함
    df_markets = pd.read_csv('market_equity_kospi.csv',index_col=0) #이는 매일 존재
   
    return df_open, df_high, df_low, df_close, df_vol, df_markets

# downgrade pytorch to 2.1.0

In [4]:
from torch_geometric.nn import GCNConv

일단 자산 풀은 30개 정도로 가져가보자.

In [5]:
from torch_geometric.nn import GCNConv


In [6]:
#torch-scatter  version 확인
import torch
torch.cuda.is_available()

True

In [7]:
df_open, df_high, df_low, df_close, df_vol, df_markets = load_data()

In [8]:
#시가총액 기준으로 상위 50개 뽑기(동적인 자산풀 운영)
selected_stocks = df_markets.loc['2020-01-31'].sort_values(ascending=False)[:50].index 

rebal_date = pd.to_datetime('2020-01-31')
rebal_date_index = df_close.index.get_loc(rebal_date)
valid_stocks = []
for stock in selected_stocks:
    if (df_open[stock].iloc[rebal_date_index-19:rebal_date_index+1].notnull().all() and
        df_high[stock].iloc[rebal_date_index-19:rebal_date_index+1].notnull().all() and
        df_low[stock].iloc[rebal_date_index-19:rebal_date_index+1].notnull().all() and
        df_close[stock].iloc[rebal_date_index-19:rebal_date_index+1].notnull().all() and
        df_vol[stock].iloc[rebal_date_index-19:rebal_date_index+1].notnull().all()):
        valid_stocks.append(stock)

# 다시 재정렬하고 30개를 뽑아서 selected_stocks_final을 만들기

selected_stocks_final = df_markets.loc['2020-01-31', valid_stocks].sort_values(ascending=False)[:30].index.tolist()

#2020년부터 2022년까지 데이터 사용
df_open_temp = df_open['2020-01-01':'2022-01-01']
df_high_temp = df_high['2020-01-01':'2022-01-01']
df_low_temp = df_low['2020-01-01':'2022-01-01']
df_close_temp = df_close['2020-01-01':'2022-01-01']
df_vol_temp = df_vol['2020-01-01':'2022-01-01']
# length는 다를 수 있음!!
df_markets = df_markets['2020-01-01':'2022-01-01']
#Tensor로 한꺼번에 구축하는 것도 방법-> 차후 고민
temp_df = pd.DataFrame(index=selected_stocks, columns = ['open', 'high', 'low', 'close', 'volume']) # 30*5 형태. 각 날짜별로 이는 구축됨.

open_values = df_open_temp.loc[rebal_date, selected_stocks]
high_values = df_high_temp.loc[rebal_date, selected_stocks]
low_values = df_low_temp.loc[rebal_date, selected_stocks]
close_values = df_close_temp.loc[rebal_date, selected_stocks]
vol_values = df_vol_temp.loc[rebal_date, selected_stocks]

temp_df['open'] = open_values.values
temp_df['high'] = high_values.values
temp_df['low'] = low_values.values
temp_df['close'] = close_values.values
temp_df['volume'] = vol_values.values




corr_df = df_close.iloc[rebal_date_index-19:rebal_date_index+1,:] #인덱싱 주의, 종가 기준으로 상관계수 matrix 생성
corr_df = corr_df[selected_stocks_final]
corrs = corr_df.corr()
corr_matrix = np.array(corrs)




def get_network_PMFG(corr_matrix):

    # Get the list of decreasing weighted links
    rholist = []
    n = len(corr_matrix)
    for i in range(n):
        for j in range(n):
            if i < j and corr_matrix[i][j] != 0:
                rholist.append([abs(float(corr_matrix[i][j])), i, j])

    rholist.sort(key=lambda x: x[0])
    rholist.reverse()

    m = len(rholist)
    filtered_matr = np.zeros((n, n))
    control = 0

    # Use tqdm for progress display
    for t in tqdm(range(m)):
        if control <= 3 * (n - 2) - 1:
            i = rholist[t][1]
            j = rholist[t][2]
            filtered_matr[i][j] = rholist[t][0]

            # Check planarity here
            G = nx.Graph()
            for k in range(n):
                for l in range(n):
                    if filtered_matr[k][l] != 0:
                        G.add_edge(k, l, weight=filtered_matr[k][l])

            if not planarity.is_planar(G):
                filtered_matr[i][j] = 0
                control += 1

    # Build the PMFG network
    PMFG = nx.Graph()
    for i in range(n):
        for j in range(n):
            if filtered_matr[i][j] != 0:
                PMFG.add_edge(i, j, weight=filtered_matr[i][j])

    return PMFG

#turn it as a torch_geometric graph
G = get_network_PMFG(corr_matrix)

# 그래프 feature 만들어 주기
for node in G.nodes:
    open_price = temp_df.iloc[node,0] #open
    G.nodes[node]['open'] = open_price
    high_price = temp_df.iloc[node,1] #high
    G.nodes[node]['high'] = high_price
    low_price = temp_df.iloc[node,2] #low
    G.nodes[node]['low'] = low_price
    close_price = temp_df.iloc[node,3] #close
    G.nodes[node]['close'] = close_price
    volume_price = temp_df.iloc[node,4] #volume
    G.nodes[node]['volume'] = volume_price


label_close = df_close[selected_stocks_final].iloc[rebal_date_index+20,:].values 

close_price = df_close[selected_stocks_final].iloc[rebal_date_index,:].values


# 정답값 만들어주기 #
label_df = pd.DataFrame(columns = selected_stocks_final, index = ['label'])
label_list = []
for stock in range(len(selected_stocks_final)):
    if label_close[stock] > close_price[stock]:
        label_list.append(1)
    else:
        label_list.append(0)
        

label_df.loc['label',:] = label_list #이후에 G의 Label값으로 넣어주기.

100%|██████████| 435/435 [00:00<00:00, 6489.38it/s]


GCN으로 hidden 만들기

In [9]:
import torch
from torch_geometric.data import Data
label_df = label_df.astype(int)
def nx_to_pyg(G, label_df):
    # Extracting node features
    x = torch.tensor([list(G.nodes[node].values()) for node in G.nodes], dtype=torch.float)
    
    # Extracting edge indices
    edge_index = torch.tensor([[e[0], e[1]] for e in G.edges], dtype=torch.long).t().contiguous()
    
    # Extracting edge weights
    edge_weights = torch.tensor([G[u][v]['weight'] for u, v in G.edges], dtype=torch.float)
    
    # Extracting node labels
    y = torch.tensor(label_df.values[0], dtype=torch.float)
    
    # Creating PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index, edge_weight=edge_weights, y=y)
    
    return data

# Assuming G is the NetworkX graph and label_df is the DataFrame containing labels
pyg_G = nx_to_pyg(G, label_df)


In [10]:
torch.__version__

'2.1.0'

In [11]:
#check cuda version from torch
torch.version.cuda

'12.1'

In [12]:
#python version
import sys
print(sys.version)

3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]


In [18]:
import torch_geometric


In [13]:
# 데이터 전처리

from torch.utils.data import Dataset, DataLoader

class StockDataset(Dataset):
    def __init__(self, df_close, df_markets, temp_df, selected_stocks_final):
        self.df_close = df_close
        self.df_markets = df_markets
        self.temp_df = temp_df
        self.selected_stocks_final = selected_stocks_final
        
    def __len__(self):
        return len(self.df_close) - 20  # 이전 시점 데이터를 사용하므로 마지막 20일은 제외
        
    def __getitem__(self, idx):
        rebal_date = self.df_close.index[idx + 20]  # 현재 날짜 (rebal_date)
        rebal_date_index = self.df_close.index.get_loc(rebal_date)
        
        # 현재 날짜의 label
        label_close = self.df_close[self.selected_stocks_final].iloc[idx + 20].values
        close_price = self.df_close[self.selected_stocks_final].iloc[idx].values
        label_list = [1 if label > close else 0 for label, close in zip(label_close, close_price)]
        label = torch.tensor(label_list, dtype=torch.float32)
        
        # 현재 날짜의 그래프 feature
        G = get_network_PMFG(corr_matrix)  # 이전과 동일한 방식으로 그래프 생성
        for node in G.nodes:
            open_price = self.temp_df.iloc[node, 0]  # open
            G.nodes[node]['open'] = open_price
            high_price = self.temp_df.iloc[node, 1]  # high
            G.nodes[node]['high'] = high_price
            low_price = self.temp_df.iloc[node, 2]  # low
            G.nodes[node]['low'] = low_price
            close_price = self.temp_df.iloc[node, 3]  # close
            G.nodes[node]['close'] = close_price
            volume_price = self.temp_df.iloc[node, 4]  # volume
            G.nodes[node]['volume'] = volume_price
        
        # 이전 시점의 그래프 feature
        prev_graphs = []
        for i in range(5):
            prev_date = self.df_close.index[idx + 20 - i - 1]
            prev_temp_df = self.temp_df.loc[:, ['open', 'high', 'low', 'close', 'volume']].loc[prev_date]
            prev_graph = get_network_PMFG(corr_matrix, prev_temp_df)
            prev_graphs.append(prev_graph)
        
        return G, prev_graphs, label


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [17]:
device.type

'cuda'

In [22]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch_geometric
import torch.nn as nn
import torch.optim as optim

# GCN 모델 정의
class GCNModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, hidden_size)
    
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# GCN을 적용한 LSTM 모델 정의
class GCNLSTMModel(nn.Module):
    def __init__(self, gcn_input_size, gcn_hidden_size, lstm_input_size, lstm_hidden_size, num_layers, num_classes):
        super(GCNLSTMModel, self).__init__()
        self.gcn_model = GCNModel(gcn_input_size, gcn_hidden_size)
        self.lstm = nn.LSTM(lstm_input_size, lstm_hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_size, num_classes)
    
    def forward(self, data, prev_data):
        gcn_output = self.gcn_model(data)
        lstm_input = torch.cat((gcn_output.unsqueeze(0), prev_data), dim=0)
        lstm_input = lstm_input.permute(1, 0, 2)  # LSTM의 입력 형태에 맞게 변환
        lstm_output, _ = self.lstm(lstm_input)
        out = self.fc(lstm_output[:, -1, :])
        return out

# 하이퍼파라미터 설정
gcn_input_size = 5  # 각 노드의 feature 개수
gcn_hidden_size = 64
lstm_input_size = gcn_hidden_size
lstm_hidden_size = 128
num_layers = 2
num_classes = 2
learning_rate = 0.001
batch_size = 32
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')





# 데이터셋 및 데이터로더 생성 (이전과 동일)
dataset = StockDataset(df_close, df_markets, temp_df, selected_stocks_final)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 모델 초기화
model = GCNLSTMModel(gcn_input_size, gcn_hidden_size, lstm_input_size, lstm_hidden_size, num_layers, num_classes).to(device)

# 손실 및 최적화 함수 (이전과 동일)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 학습 (이전과 동일)
for epoch in range(num_epochs):
    for graphs, prev_graphs, labels in train_loader:
        graphs = [g.to(device) for g in graphs]
        prev_graphs = [[g.to(device) for g in prev] for prev in prev_graphs]
        labels = labels.to(device)
        
        # 그래프 및 이전 시점 그래프를 GCN 모델에 입력할 수 있는 형태로 변환
        data_list = [torch_geometric.data.Data(x=g.nodes[node].values(), edge_index=nx.to_numpy_array(g)) for g in graphs]
        prev_data_list = [[[torch.tensor(list(prev_g.nodes[node].values())).unsqueeze(0) for node in prev_g.nodes] for prev_g in prev_graphs_single] for prev_graphs_single in prev_graphs]
        
        # Forward pass
        outputs = model(data_list, prev_data_list)
        loss = criterion(outputs, labels.long())
        
        # Backward 및 optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 학습 결과 출력
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

print('학습 완료')


100%|██████████| 435/435 [00:00<00:00, 10609.91it/s]


KeyError: Timestamp('2006-08-16 00:00:00')

In [13]:
import torch
import torch.nn.functional as F
from torch_geometric.nn.conv import GCNConv

class GCN_LSTM(torch.nn.Module):
    def __init__(self, input_dim=5, hidden_dim=16, output_dim=2):
        super(GCN_LSTM, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lstm = torch.nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # GCN 실행
        x = self.conv1(x, edge_index)
        x = F.gelu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        # LSTM 입력을 위해 차원 변환
        x = x.unsqueeze(0)  # 배치 차원 추가
        x, _ = self.lstm(x)  # LSTM 실행
        x = x.squeeze(0)  # 배치 차원 제거

        # 최종 예측
        x = self.fc(x)
        return F.log_softmax(x, dim=-1)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader

class GCN_LSTM(torch.nn.Module):
    def __init__(self, input_dim=5, hidden_dim=16, output_dim=2, num_nodes=30):
        super(GCN_LSTM, self).__init__()
        self.num_nodes = num_nodes
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lstm = torch.nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        # GCN 실행
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.gelu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        # 각 시간 단계별로 hidden representation을 저장할 리스트
        hidden_representations = []
        
        # 데이터를 시간 단계별로 분할하여 각 시간 단계에 대한 부분 그래프 생성
        for t in range(data.num_time_steps):
            # 각 시간 단계에 해당하는 노드들의 인덱스 선택
            node_indices = torch.arange(t * self.num_nodes, (t + 1) * self.num_nodes)
            
            # 해당 시간 단계에 해당하는 부분 그래프 생성
            subgraph = Data(x=x[node_indices], edge_index=edge_index)
            
            # GCN을 통해 부분 그래프에 대한 hidden representation 계산
            subgraph_representation = self.conv2(subgraph.x, subgraph.edge_index)
            hidden_representations.append(subgraph_representation)

        # LSTM의 입력 형태로 hidden representation 변환
        hidden_representations = torch.stack(hidden_representations, dim=0)  # (num_time_steps, num_nodes, hidden_dim)
        
        # LSTM 실행
        lstm_out, _ = self.lstm(hidden_representations)

        # 최종 예측
        final_hidden_state = lstm_out[:, -1, :]  # LSTM의 마지막 시간 단계의 hidden state를 사용
        x = self.fc(final_hidden_state)
        return F.log_softmax(x, dim=-1)

In [14]:
GCN_LSTM_model = GCN_LSTM()
print(GCN_LSTM_model)


GCN_LSTM(
  (conv1): GCNConv(5, 16)
  (conv2): GCNConv(16, 16)
  (lstm): LSTM(16, 16, batch_first=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)


In [15]:
#input the data to the model
output = GCN_LSTM_model(pyg_G)

In [17]:
output.shape

torch.Size([30, 2])